In [1]:
!pip install gradio

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image as im                   #load image
from tensorflow.keras.applications.resnet50 import preprocess_input      #preprocess image

In [3]:
def create_model():
    resnet50 = keras.applications.resnet50
    
    conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

    for layer in conv_model.layers:
        layer.trainable = False

    x = keras.layers.GlobalAveragePooling2D()(conv_model.output)
    x = keras.layers.Dense(512, activation='relu')(x)
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.2)(x)
    predictions = keras.layers.Dense(3, activation='softmax')(x)

    full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

    return full_model

In [4]:
import gradio as gr
import numpy as np

labels = ['Balut/Penoy','Salted egg','Table egg']

def sepia(img):
    
    img = img.reshape((-1, 224, 224, 3))

    full_model = create_model()
    
    save_dir = "C:/Users/user/Desktop/Model/saved_finalmodel_1"
    
    # Load the best model instance to evaluate the performance of the model
    full_model.load_weights(save_dir+"/"+ "finalmodel_1.h5")
    
    predictionu = full_model.predict(img, verbose=0).flatten() 
    #print(predictionu)
    
    return {labels[i]: float(predictionu[i]) for i in range(len(labels))}

iface = gr.Interface(sepia, gr.inputs.Image(shape = (224, 224)), gr.outputs.Label(num_top_classes = 3))
iface.launch()

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)